In [30]:
import pandas as pd
import numpy as np

from consts import get_region_from_city


pd.set_option("display.float_format", lambda x: "%.2f" % x)

In [31]:
q3_demand = pd.read_excel("sheets/qtr_5/q3-q4 demand.xlsx", sheet_name="qtr-3")
q4_demand = pd.read_excel("sheets/qtr_5/q3-q4 demand.xlsx", sheet_name="qtr-4")

m_q3_demand = q3_demand.set_index(["Brand","Company", "City", "Region" ])
m_q4_demand = q4_demand.set_index(["Brand", "Company", "City", "Region"])


q3_grouped = m_q3_demand.groupby(["Company", "Region"]).sum()

interesting_columns = ["Youth",	"Work",	"Recreation",	"Mountain",	"Speed"]
combined_view = m_q4_demand.join(m_q3_demand, lsuffix="_q4", rsuffix="_q3")
q4_cols, q3_cols= tuple(map(lambda q: [col+q for col in interesting_columns], ["_q4", "_q3"]))

combined_view[[*q4_cols, *q3_cols]] = combined_view[[*q4_cols, *q3_cols]].fillna(0)
trend_df = pd.DataFrame()

for indx, col in enumerate(q4_cols):
    trend_df[f"{interesting_columns[indx]}"] = combined_view[q4_cols[indx]] - combined_view[q3_cols[indx]]


trend_df

,,,,Youth,Work,Recreation,Mountain,Speed
Brand,Company,City,Region,,,,,
NS5 Rocky Pro,NS5 Bikes,Portland,NORAM,0.00,0.00,3.00,39.00,0.00
NS5 Leisure Pro,NS5 Bikes,Portland,NORAM,0.00,2.00,7.00,0.00,0.00
NS5 Rocky LTE,NS5 Bikes,Portland,NORAM,0.00,0.00,1.00,3.00,0.00
NS5 Leisure LTE,NS5 Bikes,Portland,NORAM,0.00,6.00,21.00,0.00,0.00
SR Racer,SculptRide,Portland,NORAM,0.00,0.00,0.00,0.00,81.00
...,...,...,...,...,...,...,...,...
Hustler,Novobik,Hangzhou,APAC,0.00,2.00,0.00,0.00,0.00
Wandermate,Novobik,Hangzhou,APAC,0.00,1.00,0.00,0.00,0.00
HustlerPro,Novobik,Hangzhou,APAC,0.00,2.00,0.00,0.00,0.00


In [32]:
grouped_trend_df = trend_df.groupby(["Company", "Region"]).sum()
grouped_trend_df

Youth   Work  Recreation  Mountain  Speed
Company           Region                                           
3D-X              EUROPE   1.00  79.00      118.00      1.00  81.00
ELITE FORMA BIKES APAC     0.00  95.00       35.00     19.00  49.00
                  EUROPE   1.00 217.00       96.00     25.00  76.00
NS5 Bikes         NORAM    0.00 117.00      249.00    250.00   0.00
Novobik           APAC     0.00  86.00       22.00      4.00  91.00
SculptRide        NORAM    0.00 286.00      357.00    246.00 268.00
VeloMesh          APAC     0.00 157.00       57.00     30.00   0.00
                  NORAM    0.00 106.00       47.00     24.00   0.00

In [33]:
# change all zero to 1, to not cause divide by 0 errors:
for col in interesting_columns:
    q3_grouped[col] = np.where(q3_grouped[col] == 0, 1, q3_grouped[col])
q3_grouped

,,Youth,Work,Recreation,Mountain,Speed
Company,Region,,,,,
NS5 Bikes,NORAM,1,62,132,108,1
Novobik,APAC,1,132,30,4,1
SculptRide,NORAM,1,1,7,159,28
VeloMesh,APAC,1,118,32,24,1


In [38]:
percent_increment = (grouped_trend_df/q3_grouped)
percent_increment

Youth   Work  Recreation  Mountain  Speed
Company           Region                                           
3D-X              EUROPE    NaN    NaN         NaN       NaN    NaN
ELITE FORMA BIKES APAC      NaN    NaN         NaN       NaN    NaN
                  EUROPE    NaN    NaN         NaN       NaN    NaN
NS5 Bikes         NORAM    0.00   1.89        1.89      2.31   0.00
Novobik           APAC     0.00   0.65        0.73      1.00  91.00
SculptRide        NORAM    0.00 286.00       51.00      1.55   9.57
VeloMesh          APAC     0.00   1.33        1.78      1.25   0.00
                  NORAM     NaN    NaN         NaN       NaN    NaN